In [42]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

# --- 1. Core Logic (The Requirement) ---
def sigmoid(x):
    """Sigmoid activation function"""
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    """Derivative of the sigmoid function"""
    return x * (1 - x)

class NeuralNetworkLab:
    def __init__(self):
        # --- UI Components ---
        self.title = widgets.HTML("<h2>🕸️ NumPy Neural Network: XOR Problem</h2>")
        self.description = widgets.HTML(
            "<p>Implements a single-layer perceptron with backpropagation to learn the XOR pattern.</p>"
        )
        
        # Hyperparameters
        self.lr_slider = widgets.FloatSlider(
            value=0.1, min=0.01, max=1.0, step=0.01,
            description='Learning Rate:',
            continuous_update=False
        )
        
        self.epoch_slider = widgets.IntSlider(
            value=10000, min=1000, max=50000, step=1000,
            description='Epochs:',
            continuous_update=False
        )
        
        # Controls
        self.btn_train = widgets.Button(
            description="Train Network",
            button_style='primary',
            icon='rocket',
            layout=widgets.Layout(width='200px')
        )
        
        self.btn_train.on_click(self.run_training)
        
        # Output Area
        self.output_area = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '10px'})

        # --- Layout ---
        self.ui = widgets.VBox([
            self.title,
            self.description,
            widgets.HBox([self.lr_slider, self.epoch_slider]),
            self.btn_train,
            self.output_area
        ])
        
    def display(self):
        display(self.ui)

    def run_training(self, b):
        with self.output_area:
            clear_output(wait=True)
            print("🚀 Training started...")
            
            # --- Get Hyperparameters ---
            learning_rate = self.lr_slider.value
            epochs = self.epoch_slider.value
            
            # --- Initialize Data (XOR) ---
            inputs = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
            outputs = np.array([[0], [1], [1], [0]])

            # --- Initialize Weights ---
            np.random.seed(1)
            weights = 2 * np.random.random((2, 1)) - 1
            bias = np.random.random(1)
            
            error_history = []

            # --- Training Loop (The Requirement Logic) ---
            start_time = time.time()
            
            # Progress bar for feedback
            prog_bar = widgets.IntProgress(min=0, max=epochs, description='Progress:')
            display(prog_bar)
            
            for epoch in range(epochs):
                # Forward propagation
                input_layer = inputs
                output_layer = sigmoid(np.dot(input_layer, weights) + bias)
                
                # Calculate error
                error = outputs - output_layer
                
                # Record error for plotting
                mean_error = np.mean(np.abs(error))
                error_history.append(mean_error)
                
                # Backpropagation
                adjustments = error * sigmoid_derivative(output_layer)
                weights += np.dot(input_layer.T, adjustments) * learning_rate
                bias += np.sum(adjustments) * learning_rate
                
                # Update progress bar every few steps to keep UI responsive
                if epoch % (epochs // 100) == 0:
                    prog_bar.value = epoch

            prog_bar.value = epochs
            duration = time.time() - start_time
            print(f"✅ Training completed in {duration:.2f} seconds.")
            
            # --- Visualization ---
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
            
            # 1. Loss Curve
            ax1.plot(error_history, color='purple')
            ax1.set_title("Training Error over Time")
            ax1.set_xlabel("Epochs")
            ax1.set_ylabel("Mean Absolute Error")
            ax1.grid(True, alpha=0.3)
            
            # 2. Prediction Table
            # Render predictions nicely
            col_labels = ['Input A', 'Input B', 'Target', 'Prediction']
            table_data = []
            for i in range(len(inputs)):
                table_data.append([
                    inputs[i][0], 
                    inputs[i][1], 
                    outputs[i][0], 
                    f"{output_layer[i][0]:.4f}"
                ])
            
            ax2.axis('tight')
            ax2.axis('off')
            ax2.set_title("Final Predictions")
            table = ax2.table(cellText=table_data, colLabels=col_labels, loc='center', cellLoc='center')
            table.scale(1, 2)
            table.auto_set_font_size(False)
            table.set_fontsize(10)
            
            plt.tight_layout()
            plt.show()
            
            # --- Explanation ---
            print("-" * 60)
            print("Note on XOR:")
            print("You may notice predictions are not perfect (e.g., they hover around 0.5).")
            print("This is expected: A single-layer perceptron cannot perfectly solve XOR")
            print("because the data is not linearly separable. A hidden layer is needed for perfection.")
            print("-" * 60)

# --- Run the App ---
if __name__ == "__main__":
    app = NeuralNetworkLab()
    app.display()